In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
train_datasets = tfds.load('cifar10', split='train')
valid_datasets = tfds.load('cifar10', split='test')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteFJVM6U/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteFJVM6U/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
for data in train_datasets.take(5):
  print(data['image'].shape)
  #print(data['label'])
  image = tf.cast ( data['image'], tf.float32 )/ 255.0
  label = data['label']


(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)


In [4]:
def preprocessing(data):
  image = tf.cast ( data['image'], tf.float32 )/ 255.0
  label = data['label']
  return image, label

In [5]:
BATCH_SIZE = 128
train_data = train_datasets.map(preprocessing).shuffle(1000).batch(BATCH_SIZE)
valid_data = valid_datasets.map(preprocessing).shuffle(1000).batch(BATCH_SIZE)

In [6]:
for image, label in train_data.take(1):
  print(image.shape)
# batchsize가 128이며, 한장당 32*32*3짜리 이미지이다.

(128, 32, 32, 3)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
# 모델을 작성하는 다른 방법1
model = Sequential([
    Conv2D(32,3,activation='relu', input_shape=[32,32,3]),
    MaxPooling2D(2,2),
    Conv2D(64,3,activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax'),
])

In [8]:
# 모델을 작성하는 다른 방법2 (반복문으로 레이어추가가능)
model = Sequential()
model.add(Conv2D(32,3,activation='relu', input_shape=[32,32,3]))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,3,activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [9]:
# 모델을 작성하는 다른 방법3 (레이어별 커스텀가능)
  # 나중에 복잡한 모델을 만들때 이런 functional API로 해야된다.
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_ = Input(shape=(32,32,3))
x = Conv2D(32,3, activation='relu')(input_)
x = MaxPooling2D(2,2)(x)
x = Conv2D(64,3, activation='relu')(x)
x = MaxPooling2D(2,2)(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model = Model(input_, x)

In [10]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [11]:
# 모델학습방법1
model.fit(train_data, validation_data=(valid_data), epochs=10)

Epoch 1/10
391/391 [==============================] - 22s 29ms/step - loss: 1.6635 - acc: 0.3995 - val_loss: 1.4081 - val_acc: 0.4969
Epoch 2/10
391/391 [==============================] - 4s 10ms/step - loss: 1.3023 - acc: 0.5402 - val_loss: 1.2097 - val_acc: 0.5740
Epoch 3/10
391/391 [==============================] - 4s 9ms/step - loss: 1.1658 - acc: 0.5898 - val_loss: 1.1213 - val_acc: 0.6044
Epoch 4/10
391/391 [==============================] - 5s 11ms/step - loss: 1.0680 - acc: 0.6278 - val_loss: 1.0485 - val_acc: 0.6355
Epoch 5/10
391/391 [==============================] - 4s 10ms/step - loss: 1.0028 - acc: 0.6536 - val_loss: 1.0216 - val_acc: 0.6495
Epoch 6/10
391/391 [==============================] - 5s 13ms/step - loss: 0.9527 - acc: 0.6710 - val_loss: 0.9612 - val_acc: 0.6669
Epoch 7/10
391/391 [==============================] - 5s 13ms/step - loss: 0.9069 - acc: 0.6869 - val_loss: 0.9642 - val_acc: 0.6698
Epoch 8/10
391/391 [==============================] - 5s 13ms/step - 

In [12]:
# 모델학습방법2 _ BATCH단위학습(고급학습방)
EPOCHS = 10
for epoch in range(EPOCHS):
  for batch, (image, label) in train_data.enumerate(): #128개의 batch수만큼 이미지랑 레이블을 묶는다
    loss = model.train_on_batch(image,label)
    print(f'epoch:{epoch+1}, batch:{batch+1}, loss:{loss[0]:.3f}, acc:{loss[1]:.2f}')



epoch:1, batch:1, loss:0.905, acc:0.72
epoch:1, batch:2, loss:0.640, acc:0.76
epoch:1, batch:3, loss:0.869, acc:0.73
epoch:1, batch:4, loss:0.739, acc:0.76
epoch:1, batch:5, loss:0.823, acc:0.73
epoch:1, batch:6, loss:0.698, acc:0.76
epoch:1, batch:7, loss:0.840, acc:0.67
epoch:1, batch:8, loss:0.879, acc:0.72
epoch:1, batch:9, loss:0.845, acc:0.73
epoch:1, batch:10, loss:0.753, acc:0.76
epoch:1, batch:11, loss:0.914, acc:0.72
epoch:1, batch:12, loss:0.998, acc:0.66
epoch:1, batch:13, loss:0.839, acc:0.70
epoch:1, batch:14, loss:0.809, acc:0.73
epoch:1, batch:15, loss:0.911, acc:0.70
epoch:1, batch:16, loss:0.671, acc:0.80
epoch:1, batch:17, loss:0.826, acc:0.72
epoch:1, batch:18, loss:0.778, acc:0.74
epoch:1, batch:19, loss:0.825, acc:0.73
epoch:1, batch:20, loss:0.807, acc:0.68
epoch:1, batch:21, loss:0.705, acc:0.73
epoch:1, batch:22, loss:0.790, acc:0.71
epoch:1, batch:23, loss:0.821, acc:0.74
epoch:1, batch:24, loss:0.797, acc:0.73
epoch:1, batch:25, loss:0.839, acc:0.72
epoch:1, 

In [25]:
# Grident Tape 연습 _compile부터 한다. 추후 더 복잡한 학습할때 써먹는다.
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')#모든 loss의 평균
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')#모든 loss의 평균
valid_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_acc')

@tf.function  #성능을 끌어올려주는 데코레이터
def train_step(image, label):
  with tf.GradientTape() as tape: #tape에 모든 계산시이기록된다.
    prediction = model(image, training = True)
    loss = loss_function(label, prediction)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(label, prediction)

@tf.function  #성능을 끌어올려주는 데코레이터
def valid_step(image, label):

  prediction = model(image, training = False)
  loss_function(label, prediction)

  valid_loss(loss)
  valid_acc(label, prediction)


In [26]:

EPOCHS = 10
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_acc.reset_states()
  valid_loss.reset_states()
  valid_acc.reset_states()

  for image, label in train_data:
    train_step(image, label)
  for image, label in valid_data:
    valid_step(image, label)

  print(f'epoch:{epoch+1}, loss:{train_loss.result()}, acc:{train_acc.result()}, val_loss:{valid_loss.result()}, val_acc:{valid_acc.result()}')


epoch:1, loss:0.4963071346282959, acc:0.8286399841308594, val_loss:0.7036362886428833, val_acc:0.7009999752044678
epoch:2, loss:0.4743368923664093, acc:0.8375999927520752, val_loss:0.7036362886428833, val_acc:0.6972000002861023
epoch:3, loss:0.47585853934288025, acc:0.8352599740028381, val_loss:0.7036362886428833, val_acc:0.6938999891281128
epoch:4, loss:0.46126940846443176, acc:0.8415200114250183, val_loss:0.7036362886428833, val_acc:0.7016000151634216
epoch:5, loss:0.4520018994808197, acc:0.8430799841880798, val_loss:0.7036362886428833, val_acc:0.7013000249862671
epoch:6, loss:0.44572967290878296, acc:0.8468400239944458, val_loss:0.7036362886428833, val_acc:0.6963000297546387
epoch:7, loss:0.43522533774375916, acc:0.8509200215339661, val_loss:0.7036362886428833, val_acc:0.6990000009536743
epoch:8, loss:0.4269905984401703, acc:0.8525800108909607, val_loss:0.7036362886428833, val_acc:0.6992999911308289
epoch:9, loss:0.41543906927108765, acc:0.8569599986076355, val_loss:0.70363628864288